<a href="https://colab.research.google.com/github/PokeMaster16/Machine-Learning-Projects/blob/main/fcc_predict_health_costs_with_regression.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Import libraries. You may or may not use all of these.
!pip install -q git+https://github.com/tensorflow/docs
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

try:
  # %tensorflow_version only exists in Colab.
  %tensorflow_version 2.x
except Exception:
  pass
import tensorflow as tf

from tensorflow import keras
from tensorflow.keras import layers

import tensorflow_docs as tfdocs
import tensorflow_docs.plots
import tensorflow_docs.modeling

In [ ]:
# Import data
!wget https://cdn.freecodecamp.org/project-data/health-costs/insurance.csv
dataset = pd.read_csv('insurance.csv')
dataset.tail()

print(dataset.columns)
print(dataset.tail)

In [ ]:
train_dataset, test_dataset, train_labels, test_labels = train_test_split(X, y, test_size=0.2, random_state=42)

# Normalize features
scaler = StandardScaler()
train_dataset_scaled = scaler.fit_transform(train_dataset)
test_dataset_scaled = scaler.transform(test_dataset)

# Log-transform labels (ensure they are 1D)
train_labels_log = np.log(train_labels.to_numpy()).reshape(-1,)
test_labels_log = np.log(test_labels.to_numpy()).reshape(-1,)

# Build the model
model = keras.Sequential([
    layers.Dense(128, activation='relu', input_shape=[train_dataset.shape[1]]),
    layers.Dense(64, activation='relu'),
    layers.Dense(32, activation='relu'),
    layers.Dense(1)
])

model.compile(optimizer='adam', loss='mse', metrics=['mae', 'mse'])

# Early stopping callback to avoid overfitting
early_stop = keras.callbacks.EarlyStopping(monitor='val_loss', patience=10)

# Train the model using log-transformed labels
history = model.fit(
    train_dataset_scaled, train_labels_log,
    validation_split=0.2,
    epochs=300,
    callbacks=[early_stop],
    verbose=1
)

# Evaluate the model
loss, mae, mse = model.evaluate(test_dataset_scaled, test_labels_log)
print(f"MAE (log-transformed scale): {mae}")

# Predict on the test set (ensure predictions are 1D)
predictions_log = model.predict(test_dataset_scaled).flatten()  # Flatten to 1D

# Reverse log transformation
predictions = np.exp(predictions_log)  # Predictions in the original scale
test_labels_exp = np.exp(test_labels_log)  # Test labels in the original scale

# Calculate MAE on the original scale
mae_original_scale = np.mean(np.abs(predictions - test_labels_exp))
print(f"MAE (original scale): {mae_original_scale}")

# Check if MAE is under 3500
if mae_original_scale < 3500:
    print(f"Model MAE is under 3500: {mae_original_scale}")
else:
    print(f"Model MAE is above 3500: {mae_original_scale}")

In [ ]:
# RUN THIS CELL TO TEST YOUR MODEL. DO NOT MODIFY CONTENTS.
# Test model by checking how well the model generalizes using the test set.
loss, mae, mse = model.evaluate(test_dataset, test_labels, verbose=2)

print("Testing set Mean Abs Error: {:5.2f} expenses".format(mae))

if mae < 3500:
  print("You passed the challenge. Great job!")
else:
  print("The Mean Abs Error must be less than 3500. Keep trying.")

# Plot predictions.
test_predictions = model.predict(test_dataset).flatten()

a = plt.axes(aspect='equal')
plt.scatter(test_labels, test_predictions)
plt.xlabel('True values (expenses)')
plt.ylabel('Predictions (expenses)')
lims = [0, 50000]
plt.xlim(lims)
plt.ylim(lims)
_ = plt.plot(lims,lims)
